In [174]:
import pandas as pd
import os
import datetime as dt

In [178]:
data_dir = '/Users/erikagutierrez/Documents/BSE/Term_3/Nowcasting/Assignments/PSET_1/Data/'

country_codes = pd.read_csv(data_dir + 'countries_codes_and_coordinates.csv')

datasets = []
master_datasets = []
dataset_names = []
problem_datasets = []
iteration = 0

for file in os.scandir(data_dir + 'Hackmageddon/'):
    if not file.is_file():
        for excel in os.listdir(data_dir + 'Hackmageddon/' + file.name):
            try:
                if excel != '.DS_Store':
                    datasets.append(pd.read_excel(data_dir + 'Hackmageddon/' + file.name + "/" +excel))
                    dataset_names.append(excel)
            except:
                problem_datasets.append(excel)
    elif file.is_file():
        if file.name != '.DS_Store':
            datasets.append(pd.read_excel(data_dir + 'Hackmageddon/' + file.name))
            dataset_names.append(file.name)

/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py

In [180]:
problem_datasets

[]

In [181]:
columns = []
for dataset in datasets: 
    columns.extend(dataset.columns)
columns = pd.Series(columns)
print(columns.value_counts())


columns_to_extract = ['Author', 'Target', 'Description', 'Attack', 'Country', 'Date']

Author              117
Target              117
Description         117
Attack              117
Country             117
Date                 92
Link                 91
Tags                 90
Target Class         80
Attack Class         80
ID                   75
Unnamed: 11          55
Unnamed: 10          33
Unnamed: 9           32
Attack Category      28
Target Category      28
Ref                  28
Date Discovered      25
Date Reported        25
Date Occurred        25
Unnamed: 0           16
Target\nClass         8
Attack\nClass         8
Unnamed: 8            7
Unnamed: 2            7
Unnamed: 7            7
Unnamed: 6            7
Unnamed: 5            7
Unnamed: 4            7
Unnamed: 3            7
Unnamed: 1            7
Unnamed: 12           4
pef                   1
moh                   1
mohu                  1
Id                    1
Target\nCategory      1
Class                 1
tr                    1
dtype: int64


In [182]:
columns_to_extract_except = columns_to_extract.copy()
columns_to_extract_except.remove('Date')
columns_to_extract_except.extend(['Date Reported'])

In [186]:
cleaned_datasets = []
iteration = 0
bad_datasets = []

for dataset in datasets:
    
    try:
        cleaned_datasets.append(dataset[columns_to_extract])
    except:
        try:
            cleaned_datasets.append(dataset[columns_to_extract_except])
        except:
            bad_datasets.append(iteration)
    
    iteration = iteration + 1

In [187]:
print(bad_datasets)

[]


In [185]:
# run this if bad_datasets is not empty
for bad_data in bad_datasets:
    df = datasets[bad_data]
    df.columns=df.iloc[0]
    df.drop(0, axis = 0, inplace = True)
    datasets[bad_data] = df

In [188]:
cleaned_columns = []
for dataset in cleaned_datasets: 
    cleaned_columns.extend(dataset.columns)
cleaned_columns = pd.Series(cleaned_columns)
print(cleaned_columns.value_counts())

Author           124
Target           124
Description      124
Attack           124
Country          124
Date              99
Date Reported     25
dtype: int64


In [189]:
cols_to_keep = list(cleaned_columns.unique())
cols_to_keep.remove('Date Reported')
for dataset in cleaned_datasets:
    if 'Date Reported' in dataset.columns:
        dataset.rename({'Date Reported': 'Date'}, inplace = True, axis = 1)
    dataset = dataset[cols_to_keep]
    dataset['Date'] = dataset['Date'].astype(str)

/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/3901887219.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.rename({'Date Reported': 'Date'}, inplace = True, axis = 1)
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/3901887219.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.rename({'Date Reported': 'Date'}, inplace = True, axis = 1)
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/3901887219.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [399]:
cyber_attacks = pd.DataFrame(columns = cols_to_keep)
for dataset in cleaned_datasets:
    cyber_attacks = pd.concat([cyber_attacks, dataset], axis = 0)

In [400]:
iteration = 0
dates_to_drop = []
for index, row in cyber_attacks.iterrows():
    if 'Legend' in str(row['Date']) or 'hackmageddon.com' in str(row['Date']):
        dates_to_drop.append(index)
cyber_attacks.drop(index = cyber_attacks.index[dates_to_drop], inplace=True)
cyber_attacks.reset_index(inplace = True)

In [401]:
# Pick countries

for column in country_codes.columns[1:]:
    country_codes[column] = country_codes[column].apply(lambda x: x.replace('"', "").strip())

cyber_attacks = cyber_attacks[cyber_attacks['Country'].isin(list(country_codes['Alpha-2 code']))]

cyber_attacks = pd.merge(cyber_attacks, country_codes[['Alpha-2 code', 'Alpha-3 code']], left_on = 'Country', right_on = 'Alpha-2 code')

cyber_attacks.drop(['Alpha-2 code', 'Country'], axis = 1, inplace = True)

In [402]:
bad_dates = []
for index, row in cyber_attacks.iterrows():
    try:
        dt.datetime.strptime(str(row['Date']),'%Y-%m-%d %H:%M:%S')
    except:
        bad_dates.append((row['Date'], index))

In [403]:
bad_dates

[(NaT, 380),
 (NaT, 381),
 ('Sep 8 ', 1252),
 (nan, 1404),
 (nan, 1405),
 (nan, 1406),
 (nan, 1407),
 (nan, 1408),
 (nan, 1409),
 ('Sep 8 ', 4648),
 ('12/10//2016', 4880),
 ('12/10//2016', 4881),
 (nan, 5612),
 ('Jun4 ', 5862),
 (nan, 7311),
 ('12/10//2016', 7711)]

In [404]:
cyber_attacks.iloc[380:382, -2] = '2015-11-07 00:00:00'

cyber_attacks.iloc[1252, -2] = '2014-09-08 00:00:00'

cyber_attacks.iloc[1404:1410, -2] = '2014-09-20 00:00:00'

cyber_attacks.iloc[4648,-2] = '2014-09-08 00:00:00'

cyber_attacks.iloc[4880:4882, -2] = '2016-12-10 00:00:00'

cyber_attacks.iloc[5862, -2] = '2015-06-04 00:00:00'

cyber_attacks.iloc[7711, -2] = '2016-10-12 00:00:00'

cyber_attacks.iloc[4238:4240, -2] = '2022-03-31 00:00:00'

cyber_attacks.iloc[5216:5218, -2] = '2022-03-31 00:00:00'

cyber_attacks.drop(5612, inplace = True)
cyber_attacks.drop(7311, inplace = True)

cyber_attacks.reset_index(inplace = True)

In [405]:
cyber_attacks['Date'] = pd.to_datetime(cyber_attacks['Date'])
cyber_attacks['Attacks'] = 1
cyber_attacks = cyber_attacks.groupby(['Date', 'Alpha-3 code'])['Attacks'].sum().reset_index()

In [406]:
cyber_attacks['df_index'] = cyber_attacks.index #unique value for each row
cyber_attacks = cyber_attacks.set_index('Date')
start_date = cyber_attacks.index.min()
end_date = cyber_attacks.index.max()
new_index = pd.date_range(start=start_date, end=end_date, freq='D')

def make_monthly_dataset_per_country(df, country, index):
    country_cyber_attacks = cyber_attacks[cyber_attacks['Alpha-3 code'] == country]

    country_cyber_attacks = country_cyber_attacks.reindex(new_index)

    country_cyber_attacks = country_cyber_attacks.ffill()

    duplicate_indexes = list(country_cyber_attacks[country_cyber_attacks['df_index'].duplicated()].index)

    country_cyber_attacks.loc[duplicate_indexes, 'Attacks'] = 0

    country_cyber_attacks.loc[pd.Timestamp('2014-01-01 00:00:00'), 'Attacks'] = 0

    country_cyber_attacks['Month'] = country_cyber_attacks.index.month
    country_cyber_attacks['Year'] = country_cyber_attacks.index.year
    country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()

    country_cyber_attacks.df_index = country
    
    return country_cyber_attacks


monthly_cyber_attacks = pd.DataFrame(columns = ['Month', 'Year', 'Attacks', 'df_index'])

for country in cyber_attacks['Alpha-3 code'].unique():
    monthly_cyber_attacks = pd.concat([monthly_cyber_attacks, 
                                       make_monthly_dataset_per_country(cyber_attacks, country, new_index)])
    

/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. I

/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. I

/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. I

/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. I

/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. I

/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_cyber_attacks = country_cyber_attacks.groupby(['Month', 'Year']).sum().reset_index()
/var/folders/w6/q5_zm221051fpvtg0kr3dy480000gn/T/ipykernel_24370/2577066875.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. I

In [408]:
monthly_cyber_attacks.to_csv('monthly_cyber_attacks.csv')